<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Inicialização de Bibliotecas, Pastas e Foco da Camera</p>

In [1]:
import cv2
import numpy as np
import os
import time
import data_processing as dp
from collections import deque

os.makedirs("../data", exist_ok=True)
os.makedirs("../data/inputs", exist_ok=True)

for i in range(6):
    os.makedirs(f"../data/inputs/{i}", exist_ok=True)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)
cap.set(cv2.CAP_PROP_EXPOSURE, -1)

cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cap.set(cv2.CAP_PROP_FOCUS, 10)

print("Exposição:", cap.get(cv2.CAP_PROP_EXPOSURE))
print("Foco:", cap.get(cv2.CAP_PROP_FOCUS))

Exposição: -6.0
Foco: -1.0


<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Principal</p>

In [ ]:
cap = cv2.VideoCapture(0)
bbox = False
grid = True
division = 16
dp.background_subtraction.background = np.full((480, 640), 255, dtype=np.uint8)
while True:

    ret,frame_original = cap.read()

    if not ret:
        break

    frame_original = dp.mirroring(frame_original)
    frame = frame_original.copy()
    frame = dp.gray_scale(frame)
    frame = dp.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    sample,frame = dp.down_sampling(frame,division=division)
    frame = dp.binarization(frame,limiar=5)
    sample = dp.binarization(sample,limiar=5)
    bbox_points,bb_frame = dp.bounding_box(frame,frame_vizualizer=frame_original,division=division)
    vector = dp.vectorize(sample, bbox_points=bbox_points)

    print(np.sum(vector))
    print(vector.shape)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        dp.background_subtraction.background = dp.gray_scale(frame_original.copy())
        frame = np.full_like(frame_original,255)
        print('Background reset')


    cv2.imshow('Down Sampled', sample)
    cv2.imshow('Visualizer',frame)
    cv2.imshow('Original',bb_frame)
    


cap.release()
cv2.destroyAllWindows()

1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)
1131
(1770,)

<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Video REC
</p>

In [ ]:
gravador = cv2.VideoWriter('../data/.bin/asd.avi',fourcc=cv2.VideoWriter_fourcc(*'XVID'),fps=30qqqqqqqq,frameSize=(640,480))
cam = cv2.VideoCapture(0)
rec = False
while True:
    ret,original_frame = cam.read()
    original_frame = dp.mirroring(original_frame)

    if not ret:
        break
    
    frame = original_frame.copy()
    frame = dp.gray_scale(frame)
    frame = dp.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    vector,frame = dp.down_sampling(frame,division=16)
    frame = dp.binarization(frame,limiar=5)
    vector = dp.binarization(vector,limiar=5)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        dp.background_subtraction.background = dp.gray_scale(original_frame.copy())
        frame_original = np.full_like(original_frame,255)
    elif cv2.waitKey(1) & 0xFF == ord('2'):
        rec = True
        print('gravandooo...')

    if rec:
        frame = np.stack([frame]*3, axis=-1)
        gravador.write(frame)


    cv2.imshow('Original',original_frame)
    cv2.imshow('Visualizer',frame)

cam.release()
gravador.release()
cv2.destroyAllWindows()

In [4]:
def vectorize(sample, bbox_points, division= 16):

    cut_image = sample[bbox_points[0]:bbox_points[1], bbox_points[2]:bbox_points[3]]
    somas = np.sum(cut_image, axis=0)

    if np.any(somas > 0):
        center_points = np.argmax(somas)

        total_width_length = 640 // division
        total_height_length = 480 // division
        target_center = total_width_length // 2

        left_length = center_points
        right_length = (cut_image.shape[1]) - center_points

        pad_left = target_center - left_length
        pad_right = total_width_length - target_center - right_length

        vector = np.pad(
            cut_image,
            pad_width=(
                (max(total_height_length - cut_image.shape[0],0), 0),
                (max(pad_left,0), max(pad_right,0))
            ),
            mode='constant',
            constant_values=0
        )

        vector = vector // 255

        vector = vector.flatten()
    else:
        vector = np.zeros((480 // division, 640 // division), dtype=np.uint8).flatten()
        
    return vector
